# PyTorch Model Patterns with Netron Visualization

이 노트북에서는 다양한 PyTorch 모델 패턴들을 구현하고 Netron을 사용하여 시각화해보겠습니다.

## 필요한 라이브러리 설치 및 임포트

In [1]:
# 필요한 패키지 설치
# !pip install torch torchvision onnx netron

import torch
import torch.nn as nn
import torch.onnx
import netron
import os
from IPython.display import IFrame, display
import warnings
warnings.filterwarnings('ignore')

## 유틸리티 함수: 모델 시각화

In [2]:
def visualize_model(model, input_shape, model_name):
    """
    PyTorch 모델을 ONNX로 변환하고 Netron으로 시각화
    
    Args:
        model: PyTorch 모델
        input_shape: 입력 텐서 크기 (batch_size, channels, height, width)
        model_name: 저장할 파일명
    """
    model.eval()
    
    # 더미 입력 생성
    dummy_input = torch.randn(*input_shape)
    
    # ONNX 파일명
    onnx_path = f"{model_name}.onnx"
    
    # ONNX로 내보내기
    torch.onnx.export(
        model,
        dummy_input,
        onnx_path,
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={
            'input': {0: 'batch_size'},
            'output': {0: 'batch_size'}
        }
    )
    
    print(f"✅ {model_name} ONNX 파일이 생성되었습니다: {onnx_path}")
    
    # Netron으로 시각화 (로컬에서 실행)
    try:
        netron.start(onnx_path, browse=True, port=8080)
        print(f"🌐 Netron 서버가 시작되었습니다. 브라우저에서 http://localhost:8080 을 열어보세요.")
    except:
        print(f"📁 ONNX 파일이 저장되었습니다. Netron 앱에서 {onnx_path}를 열어보세요.")
    
    return onnx_path

## Pattern 1: Sequential Network (순차적 처리)

🚂 **기차처럼 순서대로 처리하는 가장 기본적인 패턴**

In [3]:
class SequentialNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # 🚂 기차처럼 순서대로
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)     # 순차적 특징 추출
        x = self.classifier(x)   # 순차적 분류
        return x

# 모델 생성 및 시각화
sequential_model = SequentialNet(num_classes=10)
print("📊 Sequential Network 구조:")
print(sequential_model)
print("\n" + "="*50 + "\n")

# 시각화
onnx_path = visualize_model(sequential_model, (1, 3, 32, 32), "sequential_net")

📊 Sequential Network 구조:
SequentialNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): AdaptiveAvgPool2d(output_size=(1, 1))
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=256, out_features=128, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)


✅ sequential_net ONNX 파일이 생성되었습니다: sequential_net.onnx
📁 ONNX 파일이 저장되었습니다. Netron 앱에서 sequential_net.onnx를 열어보세요.


## Pattern 2: Residual Network (잔차 연결)

🛤️ **지름길(skip connection)을 만들어 깊은 네트워크 학습을 가능하게 하는 패턴**

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        # 🛤️ 메인 경로
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # 🚁 지름길 (shortcut)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        
    def forward(self, x):
        # 🏃‍♂️ 메인 경로로 처리
        residual = self.conv1(x)
        residual = self.bn1(residual)
        residual = torch.relu(residual)
        residual = self.conv2(residual)
        residual = self.bn2(residual)
        
        # ➕ 지름길과 합치기 (핵심!)
        shortcut = self.shortcut(x)
        output = residual + shortcut
        output = torch.relu(output)
        
        return output

class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        
        # 🏗️ Residual 블록들
        self.layer1 = self._make_layer(64, 64, 2, stride=1)
        self.layer2 = self._make_layer(64, 128, 2, stride=2)
        self.layer3 = self._make_layer(128, 256, 2, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)
        
    def _make_layer(self, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride))
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        
        x = self.layer1(x)  # 🔗 잔차 연결들
        x = self.layer2(x)
        x = self.layer3(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x

# 모델 생성 및 시각화
resnet_model = ResNet(num_classes=10)
print("📊 ResNet 구조:")
print(f"총 파라미터 수: {sum(p.numel() for p in resnet_model.parameters()):,}")
print("\n" + "="*50 + "\n")

# 시각화
onnx_path = visualize_model(resnet_model, (1, 3, 32, 32), "resnet")

📊 ResNet 구조:
총 파라미터 수: 2,777,674


✅ resnet ONNX 파일이 생성되었습니다: resnet.onnx
📁 ONNX 파일이 저장되었습니다. Netron 앱에서 resnet.onnx를 열어보세요.


## Pattern 3: Branching Network (다중 경로)

🌿 **여러 갈래로 나누어 다양한 특징을 추출하는 패턴 (Inception 스타일)**

In [5]:
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3_reduce, ch3x3, ch5x5_reduce, ch5x5, pool_proj):
        super().__init__()
        
        # 🌿 Branch 1: 1x1 conv
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, ch1x1, 1),
            nn.ReLU(inplace=True)
        )
        
        # 🌿 Branch 2: 1x1 conv -> 3x3 conv
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, ch3x3_reduce, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch3x3_reduce, ch3x3, 3, padding=1),
            nn.ReLU(inplace=True)
        )
        
        # 🌿 Branch 3: 1x1 conv -> 5x5 conv
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, ch5x5_reduce, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch5x5_reduce, ch5x5, 5, padding=2),
            nn.ReLU(inplace=True)
        )
        
        # 🌿 Branch 4: 3x3 pool -> 1x1 conv
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_channels, pool_proj, 1),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        # 🍴 네 갈래로 분기
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        
        # 🤝 결과 합치기 (채널 방향으로 concatenate)
        outputs = torch.cat([branch1, branch2, branch3, branch4], dim=1)
        return outputs

class BranchingNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 64, 7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(3, stride=2, padding=1)
        
        # 🏗️ Inception 블록들
        self.inception1 = InceptionBlock(64, 64, 96, 128, 16, 32, 32)  # 출력: 256채널
        self.inception2 = InceptionBlock(256, 128, 128, 192, 32, 96, 64)  # 출력: 480채널
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(480, num_classes)
        
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.maxpool1(x)
        
        x = self.inception1(x)  # 🌳 첫 번째 분기 블록
        x = self.inception2(x)  # 🌳 두 번째 분기 블록
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x

# 모델 생성 및 시각화
branching_model = BranchingNet(num_classes=10)
print("📊 Branching Network (Inception-style) 구조:")
print(f"총 파라미터 수: {sum(p.numel() for p in branching_model.parameters()):,}")
print("\n" + "="*50 + "\n")

# 시각화
onnx_path = visualize_model(branching_model, (1, 3, 32, 32), "branching_net")

📊 Branching Network (Inception-style) 구조:
총 파라미터 수: 540,090


✅ branching_net ONNX 파일이 생성되었습니다: branching_net.onnx
📁 ONNX 파일이 저장되었습니다. Netron 앱에서 branching_net.onnx를 열어보세요.


## Pattern 4: Custom Modules (재사용 가능한 컴포넌트)

🧩 **레고 블록처럼 재사용 가능한 모듈을 만들어 조립하는 패턴**

In [6]:
class ConvBlock(nn.Module):
    """재사용 가능한 Convolution 블록"""
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.block(x)

class DepthwiseConvBlock(nn.Module):
    """MobileNet 스타일의 Depthwise Separable Convolution"""
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        # Depthwise convolution
        self.depthwise = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, stride=stride, padding=1, groups=in_channels, bias=False),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True)
        )
        
        # Pointwise convolution
        self.pointwise = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

class SEBlock(nn.Module):
    """Squeeze-and-Excitation 블록"""
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.squeeze = nn.AdaptiveAvgPool2d(1)
        self.excitation = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        b, c, _, _ = x.size()
        # Squeeze
        y = self.squeeze(x).view(b, c)
        # Excitation
        y = self.excitation(y).view(b, c, 1, 1)
        # Scale
        return x * y.expand_as(x)

class CustomModularNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        
        # 🧩 레고 블록들을 조립
        self.stem = ConvBlock(3, 32, stride=2)  # 입력 처리
        
        # 🔧 다양한 블록들 조합
        self.features = nn.Sequential(
            DepthwiseConvBlock(32, 64),
            SEBlock(64),  # Attention 메커니즘
            DepthwiseConvBlock(64, 128, stride=2),
            SEBlock(128),
            DepthwiseConvBlock(128, 256, stride=2),
            SEBlock(256),
        )
        
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x):
        x = self.stem(x)
        x = self.features(x)
        x = self.classifier(x)
        return x

# 모델 생성 및 시각화
custom_model = CustomModularNet(num_classes=10)
print("📊 Custom Modular Network 구조:")
print(f"총 파라미터 수: {sum(p.numel() for p in custom_model.parameters()):,}")
print("\n🧩 사용된 커스텀 모듈들:")
print("- ConvBlock: 기본 Conv + BN + ReLU")
print("- DepthwiseConvBlock: MobileNet 스타일 분리형 컨볼루션")
print("- SEBlock: Squeeze-and-Excitation 어텐션")
print("\n" + "="*50 + "\n")

# 시각화
onnx_path = visualize_model(custom_model, (1, 3, 32, 32), "custom_modular_net")

📊 Custom Modular Network 구조:
총 파라미터 수: 60,618

🧩 사용된 커스텀 모듈들:
- ConvBlock: 기본 Conv + BN + ReLU
- DepthwiseConvBlock: MobileNet 스타일 분리형 컨볼루션
- SEBlock: Squeeze-and-Excitation 어텐션


✅ custom_modular_net ONNX 파일이 생성되었습니다: custom_modular_net.onnx
📁 ONNX 파일이 저장되었습니다. Netron 앱에서 custom_modular_net.onnx를 열어보세요.


## Pattern 5: Conditional Processing (조건부 처리)

🎭 **훈련/평가 모드나 입력에 따라 다르게 동작하는 적응적 패턴**

In [7]:
class AdaptiveNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1)
        )
        
        # 🎭 조건부 처리를 위한 컴포넌트들
        self.dropout = nn.Dropout(0.5)
        self.batch_norm = nn.BatchNorm1d(256)
        
        # 🔀 다중 분류기 (앙상블 효과)
        self.classifier1 = nn.Linear(256, num_classes)
        self.classifier2 = nn.Linear(256, num_classes)
        self.classifier3 = nn.Linear(256, num_classes)
        
        # 📊 분류기 가중치 학습
        self.classifier_weights = nn.Parameter(torch.ones(3) / 3)
        
    def forward(self, x):
        # 특징 추출
        features = self.feature_extractor(x)
        features = torch.flatten(features, 1)
        
        # 🎯 훈련/평가 모드에 따라 다르게 처리
        if self.training:
            # 훈련 시: 드롭아웃 + 배치 정규화
            features = self.dropout(features)
            features = self.batch_norm(features)
            
            # 🎲 훈련 시에는 랜덤하게 분류기 선택
            if torch.rand(1) < 0.33:
                output = self.classifier1(features)
            elif torch.rand(1) < 0.66:
                output = self.classifier2(features)
            else:
                output = self.classifier3(features)
        else:
            # 🎯 평가 시: 앙상블 예측
            features = self.batch_norm(features)  # 드롭아웃 없음
            
            out1 = self.classifier1(features)
            out2 = self.classifier2(features)
            out3 = self.classifier3(features)
            
            # 가중 평균으로 최종 예측
            weights = torch.softmax(self.classifier_weights, dim=0)
            output = (weights[0] * out1 + 
                     weights[1] * out2 + 
                     weights[2] * out3)
        
        return output

class DynamicNet(nn.Module):
    """입력 크기에 따라 동적으로 처리하는 네트워크"""
    def __init__(self, num_classes=10):
        super().__init__()
        
        # 🔧 다양한 크기의 커널들
        self.small_kernel = nn.Conv2d(3, 64, 3, padding=1)
        self.medium_kernel = nn.Conv2d(3, 64, 5, padding=2)
        self.large_kernel = nn.Conv2d(3, 64, 7, padding=3)
        
        self.features = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1)
        )
        
        self.classifier = nn.Linear(128, num_classes)
        
    def forward(self, x):
        batch_size, channels, height, width = x.size()
        
        # 📏 입력 크기에 따라 다른 커널 사용
        if height <= 32 and width <= 32:
            # 작은 이미지: 작은 커널
            x = self.small_kernel(x)
        elif height <= 64 and width <= 64:
            # 중간 이미지: 중간 커널
            x = self.medium_kernel(x)
        else:
            # 큰 이미지: 큰 커널
            x = self.large_kernel(x)
        
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        
        return x

# 모델 생성 및 시각화
adaptive_model = AdaptiveNet(num_classes=10)
print("📊 Adaptive Network 구조:")
print(f"총 파라미터 수: {sum(p.numel() for p in adaptive_model.parameters()):,}")
print("\n🎭 조건부 처리 특징:")
print("- 훈련 시: 드롭아웃 + 랜덤 분류기 선택")
print("- 평가 시: 앙상블 예측 (3개 분류기 가중 평균)")
print("\n" + "="*50 + "\n")

# 시각화 (평가 모드로 설정)
adaptive_model.eval()
onnx_path = visualize_model(adaptive_model, (1, 3, 32, 32), "adaptive_net")

📊 Adaptive Network 구조:
총 파라미터 수: 379,041

🎭 조건부 처리 특징:
- 훈련 시: 드롭아웃 + 랜덤 분류기 선택
- 평가 시: 앙상블 예측 (3개 분류기 가중 평균)


✅ adaptive_net ONNX 파일이 생성되었습니다: adaptive_net.onnx
📁 ONNX 파일이 저장되었습니다. Netron 앱에서 adaptive_net.onnx를 열어보세요.


## 모델 비교 및 분석

In [8]:
import pandas as pd

# 모델들 리스트
models = {
    'Sequential': sequential_model,
    'ResNet': resnet_model,
    'Branching (Inception)': branching_model,
    'Custom Modular': custom_model,
    'Adaptive': adaptive_model
}

# 모델 통계 수집
model_stats = []

for name, model in models.items():
    model.eval()
    
    # 파라미터 수 계산
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    # 더미 입력으로 추론 시간 측정
    dummy_input = torch.randn(1, 3, 32, 32)
    
    import time
    start_time = time.time()
    with torch.no_grad():
        _ = model(dummy_input)
    inference_time = (time.time() - start_time) * 1000  # ms
    
    model_stats.append({
        'Model': name,
        'Total Parameters': f"{total_params:,}",
        'Trainable Parameters': f"{trainable_params:,}",
        'Inference Time (ms)': f"{inference_time:.2f}",
        'Pattern Type': {
            'Sequential': '🚂 Linear Flow',
            'ResNet': '🛤️ Skip Connections',
            'Branching (Inception)': '🌿 Multi-Path',
            'Custom Modular': '🧩 Reusable Components',
            'Adaptive': '🎭 Conditional Processing'
        }[name]
    })

# 결과 테이블
df = pd.DataFrame(model_stats)
print("📊 모델 비교 분석")
print("=" * 80)
print(df.to_string(index=False))
print("\n💡 패턴별 특징:")
print("🚂 Sequential: 가장 단순하고 이해하기 쉬운 구조")
print("🛤️ ResNet: 깊은 네트워크 학습을 위한 skip connection")
print("🌿 Branching: 다양한 스케일의 특징을 동시에 추출")
print("🧩 Custom Modular: 재사용 가능한 컴포넌트로 효율적 설계")
print("🎭 Adaptive: 상황에 맞는 동적 처리")

📊 모델 비교 분석
                Model Total Parameters Trainable Parameters Inference Time (ms)             Pattern Type
           Sequential          405,002              405,002                1.44            🚂 Linear Flow
               ResNet        2,777,674            2,777,674                7.58      🛤️ Skip Connections
Branching (Inception)          540,090              540,090                4.84             🌿 Multi-Path
       Custom Modular           60,618               60,618                1.68    🧩 Reusable Components
             Adaptive          379,041              379,041                0.75 🎭 Conditional Processing

💡 패턴별 특징:
🚂 Sequential: 가장 단순하고 이해하기 쉬운 구조
🛤️ ResNet: 깊은 네트워크 학습을 위한 skip connection
🌿 Branching: 다양한 스케일의 특징을 동시에 추출
🧩 Custom Modular: 재사용 가능한 컴포넌트로 효율적 설계
🎭 Adaptive: 상황에 맞는 동적 처리


## 실제 데이터로 테스트

In [9]:
# 간단한 테스트 데이터 생성
test_input = torch.randn(4, 3, 32, 32)  # 배치 크기 4

print("🧪 실제 데이터로 모델 테스트")
print("=" * 50)
print(f"입력 크기: {test_input.shape}")
print()

for name, model in models.items():
    model.eval()
    with torch.no_grad():
        output = model(test_input)
        print(f"📋 {name}:")
        print(f"   출력 크기: {output.shape}")
        print(f"   예측 클래스: {torch.argmax(output, dim=1).tolist()}")
        print(f"   최대 확률: {torch.softmax(output, dim=1).max(dim=1)[0].mean():.3f}")
        print()

print("✅ 모든 모델이 정상적으로 동작합니다!")
print("\n🌐 생성된 ONNX 파일들을 Netron으로 열어서 모델 구조를 시각화해보세요!")
print("📁 파일 목록:")
for filename in ['sequential_net.onnx', 'resnet.onnx', 'branching_net.onnx', 'custom_modular_net.onnx', 'adaptive_net.onnx']:
    if os.path.exists(filename):
        print(f"   ✓ {filename}")

🧪 실제 데이터로 모델 테스트
입력 크기: torch.Size([4, 3, 32, 32])

📋 Sequential:
   출력 크기: torch.Size([4, 10])
   예측 클래스: [1, 1, 1, 1]
   최대 확률: 0.110

📋 ResNet:
   출력 크기: torch.Size([4, 10])
   예측 클래스: [3, 3, 3, 3]
   최대 확률: 0.108

📋 Branching (Inception):
   출력 크기: torch.Size([4, 10])
   예측 클래스: [1, 1, 1, 1]
   최대 확률: 0.107

📋 Custom Modular:
   출력 크기: torch.Size([4, 10])
   예측 클래스: [6, 6, 6, 6]
   최대 확률: 0.105

📋 Adaptive:
   출력 크기: torch.Size([4, 10])
   예측 클래스: [1, 1, 1, 1]
   최대 확률: 0.106

✅ 모든 모델이 정상적으로 동작합니다!

🌐 생성된 ONNX 파일들을 Netron으로 열어서 모델 구조를 시각화해보세요!
📁 파일 목록:
   ✓ sequential_net.onnx
   ✓ resnet.onnx
   ✓ branching_net.onnx
   ✓ custom_modular_net.onnx
   ✓ adaptive_net.onnx


## 결론 및 요약

### 🎯 학습한 PyTorch 모델 패턴들

1. **🚂 Sequential Pattern**: 가장 기본적인 순차 처리 방식
2. **🛤️ Residual Pattern**: Skip connection으로 깊은 네트워크 학습 가능
3. **🌿 Branching Pattern**: 다중 경로로 다양한 특징 추출
4. **🧩 Custom Modular Pattern**: 재사용 가능한 컴포넌트 설계
5. **🎭 Conditional Pattern**: 상황에 맞는 적응적 처리

### 💡 핵심 인사이트

- **모든 복잡한 아키텍처는 간단한 패턴들의 조합**
- **nn.Module의 `__init__`과 `forward` 분리 설계**가 핵심
- **ONNX + Netron으로 모델 구조 시각화** 가능
- **패턴별로 장단점과 적용 상황이 다름**

### 🚀 다음 단계

1. 실제 데이터셋(CIFAR-10, ImageNet)으로 훈련해보기
2. 하이퍼파라미터 튜닝으로 성능 최적화
3. 모델 압축 및 양자화 기법 적용
4. 실제 배포를 위한 최적화 (TensorRT, ONNX Runtime 등)

**Happy Deep Learning! 🎉**